<a href="https://colab.research.google.com/github/fishfishin/CT-denoise/blob/main/ResidualCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
#import visdom
import torchvision.transforms as T
import skimage.external.tifffile
import copy
import cv2
import math
from skimage.restoration import denoise_nl_means
from google.colab import drive
"""
import tensorflow as tf
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
"""
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  
print(dev)
#vis = visdom.Visdom()

In [ ]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.conv1 = nn.Conv2d(1, 32,kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(32, 32,kernel_size=3, stride=2)
        self.conv3 = nn.Conv2d(32, 32,kernel_size=3, stride=2)
        self.conv4 = nn.Conv2d(32, 32,kernel_size=3, stride=2)
        self.deconv1 = nn.ConvTranspose2d(32, 32,kernel_size=3, stride=2)
        self.deconv2 = nn.ConvTranspose2d(32, 32,kernel_size=3, stride=2)
        self.deconv3 = nn.ConvTranspose2d(32, 32,kernel_size=3, stride=2)
        self.deconv4 = nn.ConvTranspose2d(32, 1,kernel_size=3, stride=2)

        self.opt = optim.Adam(self.parameters(), lr=0.001, weight_decay=1e-5)

    def forward(self, x):
        

        out = F.relu(self.conv1(x))
        residual_1 = out
        out = F.relu(self.conv2(out))
        residual_2 = out
        out = F.relu(self.conv3(out))
        residual_3 = out
        out = F.relu(self.conv4(out))

        out = F.relu(self.deconv1(out))
        out += residual1
        out = F.relu(self.deconv2(out))
        out += residual2
        out = F.relu(self.deconv3(out))
        out += residual3
        out = F.Tanh(self.deconv4(out))
        
        return out

    def update(self, x,target1):

        out = self.forward(x)
        self.opt.zero_grad()
        
        loss = nn.MSELoss()
        upd_ = loss(out,target)
        upd_.backward()
        
        self.opt.step()

        return upd_